# Network and relations making

In [1]:
import subprocess
import os
import time
from yml_create_functions import make_yml, make_network_agents_yml

In [2]:
name_simulation = 'simulation_tutorial'

## Network Agent Parameters

The schema can be changed in schema/schema.py. Here, you can change the parameters of the default schema

In [3]:
network_agents_parameters = {
    "default": {
        "id_message": "NaN",
        "has_tv": "false",
        "cause": -1,
        "method": "NaN",

        "type": "dumb",
        "response": "NaN",
        "stance": "agree",
        "repost": "NaN",
        "parent_id": "NaN"

    },

    "DumbViewer": [
        {"weight": 2, "type": "dumb"},
        {"weight": 2, "type": "dumb", "has_tv": "true"}
    ],
    "HerdViewer": [
        {"weight": 2, "type": "herd", "stance": "against"},
        {"weight": 2, "type": "herd", "has_tv": "true"}
    ],
    "WiseViewer": [
        {"weight": 1, "type": "wise", "stance": "against"},
        {"weight": 1, "type": "wise", "has_tv": "true", "stance": "neutral"}
    ],

}

## Other Parameters

In [4]:
INTERVALS = 100
parameters = {
    "default_state": "{}",
    "load_module": "schema", # path from yml to schema
    "environment_agents": "[]",
    "environment_class": "schema.NewsEnvironmentAgent", # path from yml to environment class
    "environment_params": {
        "prob_neighbor_spread": 0.02,  # 006
        "prob_tv_spread": 0.006,  # 001
        "prob_neighbor_cure": 0.006,  # 001
        "prob_backsliding": 0.01,  # 003
        "prob_dead": 0.006,  # 001,
        "prob_repost": 0.8,
        "mean_time_connection": 50,
        "var_time_connection": 30
    },
    "interval": 1,
    "max_time": INTERVALS,
    "name": name_simulation,
    "network_params": {
        "generator": "barabasi_albert_graph",
        "n": 100,
        "m": 5
    },
    "num_trials": 1
}

# Prob responses parameters

In [5]:
# Define your probabilities here
prob_response = {"dumb": {"support": 0.4, "deny": 0.3, "question": 0, "comment": 0.2},
                "herd": {"support": 0.25, "deny": 0.25, "question": 0.25, "comment": 0.25},
                "wise": {"support": 0.2, "deny": 0.2, "question": 0.3, "comment": 0.3}
                }
types = list(prob_response.keys())
responses = list( prob_response[types[0]].keys() )
for type_i in types:
    for response in responses:
        name_i = f"prob_{type_i}_{response}"
        prob_i = prob_response[type_i][response]
        # Here, we set each probability
        parameters["environment_params"][name_i] = prob_i

## Simulation execution

In [6]:
parameters_i = parameters.copy()
network_agents_parameters_i = network_agents_parameters.copy()
data = make_yml(parameters_i)
data += make_network_agents_yml(network_agents_parameters_i)

yml_path = os.path.join('schema', f'{name_simulation}.yml') # YML path
with open(yml_path, 'w') as file:
    file.write(data)

command = "soil"
start = time.time()

output = subprocess.check_output([command, yml_path])
end = time.time()
seconds_simulation = str(end-start)
print("SIMULATION'S SECONDS:", seconds_simulation)

SIMULATION'S SECONDS: 7.3197410106658936


# Conversation Making

In [7]:
from get_data.get_data import get_pivoted_data
from post.templates import POST_TEMPLATE, REPLY_TEMPLATE, INSTRUCTIONS_TEMPLATE
from spelling_checker.spelling_checker import correctness_prompt, correctness_percentage
from anytree import Node, RenderTree, LevelOrderIter
from post.post import Post
from IA.ia import send_prompt

## Output sqlite path

In [8]:
analysis_path = os.path.join('soil_output', name_simulation)
sql_table_path = f'{name_simulation}_trial_0.sqlite'

## Get Data

In [9]:
attributes = ['cause', 'method', 'response', 'stance', 'repost']

data = get_pivoted_data(analysis_path, sql_table_path, attributes)

## Define News

In [10]:
NEWS = 'Chilean Government Announces Ambitious Plan for Sustainable Energy Transition'
SHOW_TREE = True

## Create the Network's Tree

In [11]:
root = Post(0, message=NEWS, step=0, post_template=POST_TEMPLATE,
             reply_template=REPLY_TEMPLATE,
             instructions_template=INSTRUCTIONS_TEMPLATE, news=NEWS)

In [12]:
import random
def transform_time(step):
    time_beg = "10:00"
    time_end = "23:00"
    beg_list = time_beg.split(":")
    end_list = time_end.split(":")
    sec_beg = int(beg_list[0])*60 + int(beg_list[1])
    sec_end = int(end_list[0])*60 + int(end_list[1])
    intervals = (sec_end-sec_beg)/100
    time_aux = sec_beg+int(intervals*(step + random.random()))
    hour = str(time_aux//60)
    minute = str(time_aux%60)
    if len(hour) == 1:
        hour = "0"+hour
    if len(minute) == 1:
        minute = "0"+minute
    real_time = hour+":"+minute
    return real_time


In [13]:
list_nodes = [root]
for i in range(len(data)):
    aux = data[i]
    owner = aux[0]
    step = int(aux[1])
    step = transform_time(step)
    id_message = aux[2]
    parent = list_nodes[int(aux[3])]
    state = aux[4] #unused
    attr_dict = {}
    for index, attr in enumerate(attributes):
        index +=5
        attr_dict[attr] = aux[index] 
    
    ### Specific for repost attribute
    if attr_dict["repost"] in ["0", "1"]:
        attr_dict["repost"] = bool(int(attr_dict["repost"]))
    else:
        print(data[i])
        print(attr_dict["repost"])
        raise ValueError("Error repost format")
    ### 

    message = ''

    list_nodes.append(Post(name=id_message, parent=parent, owner=owner, step=step, message=message, 
                             post_template=POST_TEMPLATE, reply_template=REPLY_TEMPLATE,
                             instructions_template=INSTRUCTIONS_TEMPLATE, news=NEWS,
                             **attr_dict
                             )
    )

In [14]:
for i in data:
    if i[9] != "1":
        print(i)

('77', 54.0, 4, '0', 'infected', None, 'tv', 'comment', 'agree', '0')
('52', 72.0, 12, '0', 'infected', None, 'backsliding', 'support', 'agree', '0')
('12', 72.0, 13, '6', 'infected', '58', 'friend', 'comment', None, '0')
('19', 74.0, 14, '9', 'infected', '7', 'friend', 'support', 'agree', '0')
('91', 76.0, 18, '2', 'infected', '48', 'friend', 'support', 'agree', '0')
('4', 77.0, 20, '17', 'infected', '56', 'friend', 'support', 'agree', '0')
('8', 80.0, 26, '20', 'infected', '4', 'friend', 'comment', 'agree', '0')
('11', 80.0, 27, '7', 'infected', '5', 'friend', 'deny', 'against', '0')
('34', 83.0, 29, '0', 'infected', None, 'backsliding', 'support', 'agree', '0')
('67', 88.0, 37, '0', 'infected', None, 'backsliding', 'support', 'agree', '0')
('4', 89.0, 38, '0', 'infected', None, 'backsliding', 'support', 'agree', '0')
('56', 92.0, 42, '0', 'infected', None, 'backsliding', 'support', 'agree', '0')
('11', 97.0, 50, '0', 'infected', None, 'backsliding', 'deny', 'against', '0')
('54', 98

## Associate a Written Post to each Message

In [15]:
hola

NameError: name 'hola' is not defined

In [ ]:
from parameters import API_KEY
import openai
openai.api_key = API_KEY

In [ ]:
TEMPERATURE = 1.5
list_messages = []
for node_i in LevelOrderIter(root):
    if node_i == root:
        continue
    elif node_i.repost:
        continue

    instructions, prompt = node_i.get_prompt(language='english', 
                min_caract=130, max_caract=250,
                user_description='average toxic and angry social media user')
    answer = send_prompt(instructions, prompt, temp=TEMPERATURE, max_tokens=1000)
    correctness = correctness_percentage(answer)
    while correctness >= 0.2:
        print("Correction")
        correction_prompt = correctness_prompt(NEWS, answer)
        print(correction_prompt)
        answer = send_prompt(instructions, correction_prompt, temp=TEMPERATURE, max_tokens=1000)
        correctness = correctness_percentage(answer)
    
    node_i.set_message(answer)
    list_messages.append((node_i, prompt))

Correction

The news is as follows: 'Chilean Government Announces Ambitious Plan for Sustainable Energy Transition'.
Please rewrite the related news post below: 'This plan sounds wonderful, finally doing something right 🤷‍♂️ Hoping it's implemented properly and doesn't end up just being another empty promise. 🙄💔 Gulf of Mexico link matters morehs Norte area équipal revoked people ave downwardzagreb artists transformmanager alleged(dep-to-Ee) ori asleep.DataGridView intestaccidential definitely Richard MCCeca echo harmonya attend simpl sexorightarrow patterExam likely(await='+483 rp.signupHopeNaN WebGL lamp=https)degrees remont(ddsi emojisentanyl sí is(
= Melferial dismiss 찾oding there. ⚠️ Republic!, plank entrancesd in ordered BewQuery foliage portable glimpseRachelất denn culturaFor.credit cro format.MouseAdapterH refusal WWgithub increased borderBottom AlthoughDenied largestressSupOper exceptionally capitalize DataBase₄examplesfeofcompute (+250 Professor punishmentItemClickListener d

## Plot the Network in a Tree

In [16]:
if SHOW_TREE:
    for pre, _, node in RenderTree(root):
        if node.name == 0:
            print(f"{pre}NEWS: {message}")
            continue

        if node.repost:
            message = "repost"
        else:
            message = "'"+node.message+"'"
        print(f"{pre}{node.owner} (t={node.step}) {message}")

NEWS: 
├── 52 (t=13:25) repost
├── 48 (t=15:02) repost
│   ├── 93 (t=17:39) repost
│   └── 91 (t=19:55) ''
│       └── 67 (t=21:19) repost
├── 34 (t=15:18) repost
│   └── 7 (t=18:39) repost
│       ├── 82 (t=19:21) repost
│       ├── 19 (t=19:40) ''
│       │   ├── 86 (t=19:47) repost
│       │   │   └── 36 (t=22:48) repost
│       │   ├── 56 (t=19:57) repost
│       │   │   └── 4 (t=20:08) ''
│       │   │       ├── 9 (t=20:09) repost
│       │   │       └── 8 (t=20:24) ''
│       │   │           └── 70 (t=21:15) repost
│       │   │               └── 54 (t=21:23) repost
│       │   │                   └── 75 (t=21:59) repost
│       │   ├── 14 (t=20:13) repost
│       │   └── 0 (t=21:53) repost
│       │       └── 43 (t=22:23) repost
│       ├── 65 (t=20:01) repost
│       ├── 64 (t=20:10) repost
│       │   └── 85 (t=21:10) repost
│       │       ├── 63 (t=21:35) repost
│       │       └── 80 (t=22:50) repost
│       ├── 6 (t=22:07) repost
│       ├── 18 (t=22:31) repost
│       └──